#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys
import time

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import *
from src.annotator.dep_parser import *

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [2]:
# 语料库文件所在目录
data_dir = '../data/raw'

# 读取数据
# limit 参数控制平行句对数量
data = load_data(data_dir, limit=10)
print(f"成功读取 {len(data)} 条平行句对")

# 预览数据
# 第一列：汉语原文，选自《鹿鼎记》
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
print("数据前 5 行如下：")
data.head()

成功读取 10 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


#### **3. 大模型英文依存句法标注：UD 标注集**

In [3]:
# === 加载模型：LLM UD 标注集 ===

# 语种：英语
lang = 'english'

# 标注集：UD
# https://universaldependencies.org/u/dep/
# https://universaldependencies.org/treebanks/en_ewt/index.html
tagset = 'ud'

# 标注模式：LLM API
mode = 'llm'

# 大模型：deepseek-v3.2 | glm-4.7 | qwen3-max
llm_model = 'deepseek-v3.2'

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
t0 = time.time()
llm_en_ud_parser = DEPParser(
    lang=lang,
    tagset=tagset,
    mode=mode,
    llm_model=llm_model,
)
t1 = time.time()
print('LLM UD 英文依存句法标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

LLM UD 英文依存句法标注模型加载完毕！
加载耗时：1.83s


In [4]:
# === 标注译文：LLM UD 标注集 ===

# 注意：
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，再次运行程序，时间显示为 0.0 秒
llm_en_ud_annos = annotate_data(
    data['target'],
    llm_en_ud_parser,
)

# 打印标注结果
print("\n=== LLM UD 英文依存句法标注结果预览 ===")
display_anno(llm_en_ud_annos[0])

Dependency Parsing: 100%|██████████████████████████████████████████████████████████████| 10/10 [04:31<00:00, 27.19s/it]


=== LLM UD 英文依存句法标注结果预览 ===

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
[('Along', 'road', 'case'), ('a', 'road', 'det'), ('coastal', 'road', 'amod'), ('road', 'escorting', 'obl'), ('somewhere', 'road', 'advmod'), ('south', 'somewhere', 'nmod'), ('of', 'River', 'case'), ('the', 'River', 'det'), ('Yangtze', 'River', 'compound'), ('River', 'south', 'nmod'), (',', 'road', 'punct'), ('a', 'detachment', 'det'), ('detachment', 'escorting', 'nsubj'), ('of', 'soldiers', 'case'), ('soldiers', 'detachment', 'nmod'), (',', 'detachment', 'punct'), ('each', 'armed', 'nsubj'), ('of', 'them', 'case'), ('them', 'each', 'nmod'), ('armed', 'detachment', 'acl'), ('with', 'halberd', 'case'), ('a', 'halberd', 'det'), ('halberd', 'armed', 'obl'),

#### **4. 英文依存句法标注模型对比：LLM & Stanza**

In [5]:
# === 加载模型：Stanza UD 标注集 ===

# 语种：英语
lang = 'english'

# 标注集：UD
# https://universaldependencies.org/u/dep/
# https://universaldependencies.org/treebanks/en_ewt/index.html
tagset = 'ud'

# 标注模式：Stanza 本地模型
mode = 'local'

# Stanza UD 模型：combined_charlm
# https://stanfordnlp.github.io/stanza/depparse.html
# 首次加载需从网络将模型下载至本地
t0 = time.time()
stanza_en_ud_parser = DEPParser(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
t1 = time.time()
print('Stanza UD 英文依存句法标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

2026-02-08 23:48:52 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2026-02-08 23:48:52 INFO: Using device: cuda
2026-02-08 23:48:52 INFO: Loading: tokenize
2026-02-08 23:48:52 INFO: Loading: pos
2026-02-08 23:48:55 INFO: Loading: lemma
2026-02-08 23:48:56 INFO: Loading: depparse
2026-02-08 23:48:56 INFO: Done loading processors!


Stanza UD 英文依存句法标注模型加载完毕！
加载耗时：4.64s


In [6]:
# === 标注译文：Stanza UD 标注集 ===

stanza_en_ud_annos = annotate_data(
    data['target'],
    stanza_en_ud_parser,
)

# 打印标注结果
print("\n=== Stanza UD 英文依存句法标注结果预览 ===")
display_anno(stanza_en_ud_annos[0])

Dependency Parsing: 100%|██████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.63it/s]


=== Stanza UD 英文依存句法标注结果预览 ===

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
[('Along', 'road', 'case'), ('a', 'road', 'det'), ('coastal', 'road', 'amod'), ('road', 'escorting', 'obl'), ('somewhere', 'south', 'advmod'), ('south', 'road', 'advmod'), ('of', 'River', 'case'), ('the', 'River', 'det'), ('Yangtze', 'River', 'compound'), ('River', 'south', 'obl'), (',', 'road', 'punct'), ('a', 'detachment', 'det'), ('detachment', 'escorting', 'nsubj'), ('of', 'soldiers', 'case'), ('soldiers', 'detachment', 'nmod'), (',', 'each', 'punct'), ('each', 'armed', 'nsubj'), ('of', 'them', 'case'), ('them', 'each', 'nmod'), ('armed', 'detachment', 'amod'), ('with', 'halberd', 'case'), ('a', 'halberd', 'det'), ('halberd', 'armed', 'obl'), (','

In [7]:
# === 模型对比：LLM UD & Stanza UD ===

print('英文依存句法标注模型对比（UD 标注集）：LLM & Stanza')
compare_annos(
    llm_en_ud_annos,
    stanza_en_ud_annos,
    annos_1_name="LLM",
    annos_2_name="Stanza",
)

英文依存句法标注模型对比（UD 标注集）：LLM & Stanza

[ID]: 00001
Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
--------------------------------------------------------------------------------
LLM: {('south', 'somewhere', 'nmod'), ('armed', 'detachment', 'acl'), ('River', 'south', 'nmod'), ('northwards', 'trudging', 'obl:unmarked'), ('somewhere', 'road', 'advmod')}
Stanza: {('somewhere', 'south', 'advmod'), ('armed', 'detachment', 'amod'), ('northwards', 'trudging', 'advmod'), ('south', 'road', 'advmod'), ('River', 'south', 'obl')}

[ID]: 00002
In each of the first three carts a single male prisoner was caged, identifiable by his dress as a member of the scholar class.
--------------------------------------------------------------------------------
LLM: {('of', 'each', 'case'), ('each', 'carts', 'det'), ('identifiable', 'caged', 'advcl'),